## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-14-18-50-01 +0000,nyt,Hong Kong Court to Give Verdict on Media Mogul...,https://www.nytimes.com/2025/12/14/world/asia/...
1,2025-12-14-18-49-18 +0000,nyt,Live Updates: 11 Killed as Gunmen Target Hanuk...,https://www.nytimes.com/live/2025/12/14/world/...
2,2025-12-14-18-48-56 +0000,nypost,This million-dollar Picasso could be yours for...,https://nypost.com/2025/12/14/world-news/picas...
3,2025-12-14-18-48-45 +0000,cbc,"Zelenskyy offers to drop NATO aspirations, rem...",https://www.cbc.ca/news/world/ukraine-nato-pea...
4,2025-12-14-18-47-05 +0000,nypost,Iconic Rocky Balboa statue could be moving bac...,https://nypost.com/2025/12/14/us-news/iconic-r...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2343/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
48,brown,27
49,university,22
42,bondi,17
44,shooting,16
51,trump,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
7,2025-12-14-18-41-53 +0000,nypost,President Trump mourns victims of Bondi Beach ...,https://nypost.com/2025/12/14/us-news/presiden...,130
11,2025-12-14-18-36-11 +0000,wapo,Live updates: Person of interest in Brown Univ...,https://www.washingtonpost.com/nation/2025/12/...,98
133,2025-12-14-01-20-18 +0000,nypost,Terrified Brown University students describe ...,https://nypost.com/2025/12/13/us-news/terrifie...,91
28,2025-12-14-17-41-00 +0000,wsj,Police Detain Person of Interest in Deadly Bro...,https://www.wsj.com/us-news/police-detain-pers...,87
66,2025-12-14-12-31-01 +0000,cbc,Person of interest in custody in deadly Brown ...,https://www.cbc.ca/news/world/person-of-intere...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
7,130,2025-12-14-18-41-53 +0000,nypost,President Trump mourns victims of Bondi Beach ...,https://nypost.com/2025/12/14/us-news/presiden...
60,53,2025-12-14-13-40-25 +0000,cbc,LIVE UPDATES | World leaders condemn 'antisemi...,https://www.cbc.ca/news/world/livestory/hanukk...
122,33,2025-12-14-04-43-45 +0000,nypost,Video shows first glimpse of suspected Brown U...,https://nypost.com/2025/12/13/us-news/brown-un...
46,30,2025-12-14-15-50-19 +0000,nypost,Trump admits he’s not sure GOP will hold onto ...,https://nypost.com/2025/12/14/us-news/trump-ad...
67,26,2025-12-14-12-28-49 +0000,nypost,Brown U. gunman shouted something before openi...,https://nypost.com/2025/12/14/us-news/brown-un...
163,24,2025-12-13-20-03-02 +0000,nypost,"Kushner, Witkoff travel to Germany for pivotal...",https://nypost.com/2025/12/13/world-news/kushe...
101,22,2025-12-14-10-00-00 +0000,wsj,The falling out between Rep. Marjorie Taylor G...,https://www.wsj.com/politics/policy/marjorie-t...
50,21,2025-12-14-15-32-12 +0000,nypost,Bondi Beach terror survivor describes how cops...,https://nypost.com/2025/12/14/world-news/bondi...
12,18,2025-12-14-18-30-35 +0000,cbc,Poilievre says Conservatives' affordability fo...,https://www.cbc.ca/news/politics/poilievre-aff...
37,18,2025-12-14-17-00-00 +0000,wsj,"For decades, Tonka trucks have been a family s...",https://www.wsj.com/economy/trade/tonka-trucks...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
